In [ ]:
!pip install -q deltalake   --upgrade
!pip install -q duckdb      --upgrade

In [ ]:
#### use azure key vault, otherwise store data in myworkspace and don't share it with anyone
import duckdb
try:
    # input the abfss path of ini file, does not require a mounted path
    duckdb.sql(f""" CREATE or replace SECRET onelake (TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{notebookutils.credentials.getToken('storage')}') """)
    kv =duckdb.sql(""" select trim(split_part("[myvars]",' : ', 1) ) as var, trim(split_part("[myvars]",' : ', 2)) as values from
    read_csv('abfss://xxxxxxxa@onelake.dfs.fabric.microsoft.com/yyyyyyy/Files/KV/variable.ini')
    """)
except:
    ### works in local laptop or when you have a shortcut in onelake etc
    kv =duckdb.sql(""" select trim(split_part("[myvars]",' : ', 1) ) as var, trim(split_part("[myvars]",' : ', 2)) as values from
    read_csv('/lakehouse/default/Files/KV/variable.ini')
    """)

kv_values=duckdb.sql( " from kv where var in ('uc_endpoint' ,'uc') ").fetchall()
for key, value in kv_values:
    print(key)
    globals()[key] = value

uc_endpoint
uc


# Daft

In [ ]:
!pip install -q getdaft[unity]

In [4]:
from daft.unity_catalog import UnityCatalog
import daft

unity = UnityCatalog(
    endpoint= uc_endpoint,
    token=uc,
)


In [5]:
unity.list_tables("databricks_ne.aemo")

['databricks_ne.aemo.calendar',
 'databricks_ne.aemo.duid',
 'databricks_ne.aemo.mstdatetime',
 'databricks_ne.aemo.price',
 'databricks_ne.aemo.scada',
 'databricks_ne.aemo.state',
 'databricks_ne.aemo.summary']

In [6]:
unity_table = unity.load_table("databricks_ne.aemo.scada")
df = daft.read_deltalake(unity_table)
df.show()

UNITUtf8,VERSIONFloat64,"SETTLEMENTDATETimestamp(Microseconds, Some(""UTC""))",RUNNOFloat64,DUIDUtf8,INTERVENTIONFloat64,DISPATCHMODEFloat64,AGCSTATUSFloat64,INITIALMWFloat64,TOTALCLEAREDFloat64,RAMPDOWNRATEFloat64,RAMPUPRATEFloat64,LOWER5MINFloat64,LOWER60SECFloat64,LOWER6SECFloat64,RAISE5MINFloat64,RAISE60SECFloat64,RAISE6SECFloat64,MARGINAL5MINVALUEFloat64,MARGINAL60SECVALUEFloat64,MARGINAL6SECVALUEFloat64,MARGINALVALUEFloat64,VIOLATION5MINDEGREEFloat64,VIOLATION60SECDEGREEFloat64,VIOLATION6SECDEGREEFloat64,VIOLATIONDEGREEFloat64,LOWERREGFloat64,RAISEREGFloat64,AVAILABILITYFloat64,RAISE6SECFLAGSFloat64,RAISE60SECFLAGSFloat64,RAISE5MINFLAGSFloat64,RAISEREGFLAGSFloat64,LOWER6SECFLAGSFloat64,LOWER60SECFLAGSFloat64,LOWER5MINFLAGSFloat64,LOWERREGFLAGSFloat64,RAISEREGAVAILABILITYFloat64,RAISEREGENABLEMENTMAXFloat64,RAISEREGENABLEMENTMINFloat64,LOWERREGAVAILABILITYFloat64,LOWERREGENABLEMENTMAXFloat64,LOWERREGENABLEMENTMINFloat64,RAISE6SECACTUALAVAILABILITYFloat64,RAISE60SECACTUALAVAILABILITYFloat64,RAISE5MINACTUALAVAILABILITYFloat64,RAISEREGACTUALAVAILABILITYFloat64,LOWER6SECACTUALAVAILABILITYFloat64,LOWER60SECACTUALAVAILABILITYFloat64,LOWER5MINACTUALAVAILABILITYFloat64,LOWERREGACTUALAVAILABILITYFloat64,transactionIdUtf8,fileUtf8,PRIORITYInt32
None,None,2024-05-06 00:00:00 +00:00,None,ALBANY_WF1,0,None,None,0.2317222,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:05:00 +00:00,None,ALBANY_WF1,0,None,None,0.2317222,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:10:00 +00:00,None,ALBANY_WF1,0,None,None,0.2102777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:15:00 +00:00,None,ALBANY_WF1,0,None,None,0.2102777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:20:00 +00:00,None,ALBANY_WF1,0,None,None,0.2102777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:25:00 +00:00,None,ALBANY_WF1,0,None,None,0.1999444,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:30:00 +00:00,None,ALBANY_WF1,0,None,None,0.1895556,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab0d5,SCADA_2024-05-06.json,1
None,None,2024-05-06 00:35:00 +00:00,None,ALBANY_WF1,0,None,None,0.1460555,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0472088c-561e-462f-9073-781e437ab

# DuckDB does not work yet

In [ ]:
import duckdb
duckdb.sql(f"""
    INSTALL uc_catalog ;
    LOAD uc_catalog;
    CREATE SECRET (
	TYPE UC,
	TOKEN '{uc}',
	ENDPOINT '{uc_endpoint}'
);""")

In [ ]:
duckdb.sql(""" ATTACH 'databricks_ne' AS myk_catalog (TYPE UC_CATALOG) """)

In [ ]:
duckdb.sql(""" SHOW ALL TABLES;""")